# Comparison of TC hazard assessments

This notebook provides an interface to compare the TCHA 2018 with the existing benchmark hazard assessment (published in 2015). 

The new hazard assessment takes advantage of several upgrades to the underlying TCRM system, including a shift to indivisual events (enabling use of the Generalised Pareto Distribution over the Generalised Extreme Value distribution), a parametric model of radius to maximum wind and refinement of the landfall decay model to better suit observed decay rates in Australia. 



In [ ]:
%matplotlib inline
from __future__ import print_function, division

In [ ]:
import os
from os.path import join as pjoin
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.collections import LineCollection
from matplotlib.ticker import MultipleLocator
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib.lines import Line2D

import cartopy.crs as ccrs
import cartopy.feature as feature
import shapely.geometry as sg


from netCDF4 import Dataset

from Utilities.config import ConfigParser
import seaborn as sns
sns.set_context("poster")
sns.set_style("whitegrid")

Start by setting the path to the hazard data for each assessment. We're primarily interested in the AEP wind speeds, so we point to the netCDF file holding the AEP wind speed data. 

In [ ]:
tcha1_path = "/g/data/fj6/TCRM/benchmark/hazard/hazard.nc"
tcha2_path = "/g/data/fj6/TCRM/TCHA18/hazard/hazard_emp.nc"

tcha1 = Dataset(tcha1_path, 'r')
tcha2 = Dataset(tcha2_path, 'r')

Define functions to help intersect the different domains for the hazard assessments. The first TCHA had a larger extent, going well into the South Pacific Ocean (180E). 

In [ ]:
def getExtent(ncobj):
    lon = ncobj.variables['lon'][:]
    lat = ncobj.variables['lat'][:]
    return (np.min(lon), np.max(lon), np.min(lat), np.max(lat))

def intersectDomain(ncobj1, ncobj2):
    lon1 = ncobj1.variables['lon'][:]
    lat1 = ncobj1.variables['lat'][:]
    lon2 = ncobj2.variables['lon'][:]
    lat2 = ncobj2.variables['lat'][:]
    extents1 = getExtent(ncobj1)
    extents2 = getExtent(ncobj2)
    
    idx1 = np.where((lon1 <= extents2[1]) & (lon1 >= extents2[0]))[0]
    jdy1 = np.where((lat1 <= extents2[3]) & (lat1 >= extents2[2]))[0]
    idx2 = np.where((lon2 <= extents1[1]) & (lon2 >= extents1[0]))[0]
    jdy2 = np.where((lat2 <= extents1[3]) & (lat2 >= extents1[2]))[0]
    return idx1, jdy1, idx2, jdy2

In [ ]:
idx1, jdy1, idx2, jdy2 = intersectDomain(tcha1, tcha2)
lon1 = tcha1.variables['lon'][:]
lat1 = tcha1.variables['lat'][:]
lon2 = tcha2.variables['lon'][:]
lat2 = tcha2.variables['lat'][:]

In [ ]:
xgrid1, ygrid1 = np.meshgrid(lon1[idx1], lat1[jdy1])
xgrid2, ygrid2 = np.meshgrid(lon2[idx2], lat2[jdy2])

In [ ]:
wspd1 = tcha1.variables['wspd'][:, jdy1, idx1]
wspd2 = np.fliplr(tcha2.variables['wspd'][:, jdy2, idx2])
wspddiff = wspd2 - wspd1

In [ ]:
tindex = 6
ax = plt.axes(projection=ccrs.PlateCarree())
cs = ax.contourf(xgrid1, ygrid1, wspddiff[tindex, :, :], np.arange(-20, 20.1, 1.), 
                 cmap='RdBu_r', extend="both")
cl = ax.contour(xgrid1, ygrid1, wspddiff[tindex, :, :],[-15, -10, -5, 0,5, 10, 15], linewidths=0.5, colors='k')
ax.coastlines(resolution='50m', color='black', linewidth=1)
plt.colorbar(cs, orientation="horizontal", label="Wind speed (m/s)")
plt.clabel(cl, fmt="%0.1f",fontsize='xx-small')
xlocator = MultipleLocator(10)
ylocator = MultipleLocator(5)
gl = ax.gridlines(linestyle=":", color='k', draw_labels=True, xlocs=xlocator, ylocs=ylocator)
gl.xlabels_top = False
gl.ylabels_right = False
years = tcha1.variables['return_period'][:]
aep = 1./years[tindex]
ax.set_title("{0:.2f}% AEP wind speed difference\n(V2 - V1)".format(aep*100.))


Overall, a decrease in wind hazard over the ocean areas to Australia's east and west. Over Australia, the hazard is reduced, in line with the expected behaviour following refinement of the landfall decay model. There is little change in hazard over the Top End (wester part of NT), aligning with a zone of comparatively lower hazard present in both TCHA V1 and V2. 

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
cs = ax.contourf(xgrid1, ygrid1, 100.*(wspddiff[tindex, :, :]/wspd2[tindex, :, :]), np.arange(-40, 40.1, 5.), 
                 cmap='RdBu_r', extend='both')
cl = ax.contour(xgrid1, ygrid1, 100.*(wspddiff[tindex, :, :]/wspd2[tindex, :, :]),[-15, -10, -5, 0,5, 10, 15], linewidths=0.5, colors='k')
ax.coastlines(resolution='50m', color='black', linewidth=1)
plt.colorbar(cs, orientation="horizontal", label="Percentage change")
plt.clabel(cl, fmt="%0.1f",fontsize='xx-small')
xlocator = MultipleLocator(10)
ylocator = MultipleLocator(5)
gl = ax.gridlines(linestyle=":", color='k', draw_labels=True, xlocs=xlocator, ylocs=ylocator)
gl.xlabels_top = False
gl.ylabels_right = False
ax.set_title("{0:.2f}% AEP wind speed relative change\n(V2 - V1)/V2".format(aep*100.))

In [ ]:
import io
import database
import pandas as pd


from Utilities.config import ConfigParser
from Utilities.maputils import find_index
from ipywidgets import interact, fixed, Dropdown, FloatSlider, interact_manual, interactive
import ipywidgets as widgets
from IPython.display import display, Image

import seaborn as sns
sns.set_context("poster")
sns.set_style("whitegrid")

In [ ]:
configstr = """
[DataProcess]
InputFile=/home/547/cxa547/tcrm/input/Allstorms.ibtracs_wmo.v03r09.csv
Source=IBTRACS
StartSeason=1981
FilterSeasons=False

[Region]
; Domain for windfield and hazard calculation
gridLimit={'xMin':175.,'xMax':185.,'yMin':-23.0,'yMax':-13.0}
gridSpace={'x':1.0,'y':1.0}
gridInc={'x':1.0,'y':0.5}

[TrackGenerator]
NumSimulations=10000
YearsPerSimulation=1
SeasonSeed=68876543
TrackSeed=334825
TimeStep=1.0

[Input]
landmask = /home/547/cxa547/tcrm/input/landmask.nc
mslpfile = /home/547/cxa547/tcrm/MSLP/slp.day.ltm.nc
datasets = IBTRACS,LTMSLP

[Output]
Path=/g/data/fj6/TCRM/TCHA18/

[Hazard]
Years=2,5,10,20,25,50,100,200,250,500,1000,2000,2500,5000
MinimumRecords=10
CalculateCI=False

[Logging]
LogFile=/g/data/w85/TCRM_data/tcrm2.1/log/tcrm2.1.log
LogLevel=INFO
Verbose=False

[IBTRACS]
; Input data file settings
url = ftp://eclipse.ncdc.noaa.gov/pub/ibtracs/v03r06/wmo/csv/Allstorms.ibtracs_wmo.v03r09.csv.gz
path = /home/547/cxa547/tcrm/input/
filename = Allstorms.ibtracs_wmo.v03r09.csv
columns = tcserialno,season,num,skip,skip,skip,date,skip,lat,lon,skip,pressure
fielddelimiter = ,
numberofheadinglines = 3
pressureunits = hPa
lengthunits = km
dateformat = %Y-%m-%d %H:%M:%S
speedunits = kph

[LTMSLP]
; MSLP climatology file settings
URL = ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.derived/surface/slp.day.1981-2010.ltm.nc
path = /home/547/cxa547/tcrm/MSLP
filename = slp.day.ltm.nc

[Process]
DatFile=/g/data/w85/TCRM_data/tcrm-v2/process/dat/tcrm-v2.dat
ExcludePastProcessed=False
"""

In [ ]:
config = ConfigParser()
config.readfp(io.BytesIO(configstr))

db = database.HazardDatabase(configstr)
locations = db.getLocations()
locNameList = list(locations['locName'])
outputPath = config.get('Output', 'Path')
years = tcha1.variables['return_period'][:]

In [ ]:
from PlotInterface.curves import SemilogRangeCompareCurve, saveFigure
def plotHazardCurve(locName):
    locId = locations['locId'][locations['locName']==locName][0]
    locLon = locations['locLon'][locations['locId']==locId][0]
    locLat = locations['locLat'][locations['locId']==locId][0]

    i1 = find_index(lon1[idx1], locLon)
    j1 = find_index(lat1[jdy1], locLat)
    i2 = find_index(lon2[idx2], locLon)
    j2 = find_index(lat2[-jdy2], locLat)

    h1 = wspd1[:, j1, i1]
    h2 = wspd2[:, j2, i2]
    fig, ax = plt.subplots()
    ax.semilogx(years, h1, label="TCHA 2014", color="#006983")
    ax.semilogx(years, h2, label="TCHA 2018", color="#A33F1F")
    ax.set_xlabel("Average recurrence interval (years)")
    ax.set_ylabel("Wind speed (m/s)")
    ax.set_title(r"{0} ({1:.2f}$^\circ$S, {2:.2f}$^\circ$E)".format(locName, locLat, locLon))
    ax.legend(loc=2, fancybox=True)
    ax.grid(which="both", linestyle="--")
    ax.set_ylim((0, 100))
    plt.savefig(pjoin(outputPath, "plots/hazard/comparison", "{0}.png".format(locId)))
    plt.show()
    

In [ ]:
w = interactive(plotHazardCurve, 
                locName=Dropdown(options=locNameList,
                                 description="Location"),
               )
display(w)